# 3.7Softmax回归的简洁实现

In [1]:
import torch
from torch import nn
from torch.nn import init
import numpy as np
import sys
sys.path.append(".data/dive")
import d2lzh_pytorch as d2l

print(torch.__version__)

1.3.0+cpu


## 3.7.1获取和读取数据

In [3]:
batch_size = 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, root='data/dive')

## 3.7.2 定义和初始化模型

In [4]:
num_inputs = 784 
num_outputs = 10 # 图片类型：10

class LinearNet(nn.Module):
    def __init__(self, num_inputs, num_outputs):
        super(LinearNet, self).__init__()
        self.linear = nn.Linear(num_inputs, num_outputs)
    def forward(self, x): # x 的形状: (batch_size, 1, 28, 28)
        y = self.linear(x.view(x.shape[0], -1)) # x的形状：(batch_size,784)
        return y
    
# net = LinearNet(num_inputs, num_outputs)

In [5]:
class FlattenLayer(nn.Module):
    '''
    每个batch样本x的形状转换
    '''
    def __init__(self):
        super(FlattenLayer, self).__init__()
    def forward(self, x): # x 的形状: (batch, *, *, ...)
        return x.view(x.shape[0], -1)


In [6]:
from collections import OrderedDict
net = nn.Sequential(
        # FlattenLayer(),
        # LinearNet(num_inputs, num_outputs) 
        OrderedDict([
           ('flatten', FlattenLayer()),
           ('linear', nn.Linear(num_inputs, num_outputs))]) # 或者写成我们自己定义的 LinearNet(num_inputs, num_outputs) 也可以
        )

In [7]:
init.normal_(net.linear.weight, mean=0, std=0.01)
init.constant_(net.linear.bias, val=0)

Parameter containing:
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], requires_grad=True)

## 3.7.3Softmax和交叉熵损失函数

In [8]:
loss = nn.CrossEntropyLoss() 
# class torch.nn.CrossEntropyLoss(weight=None, size_average=None, ignore_index=-100, reduce=None, reduction='mean')

## 3.7.4定义优化算法

In [9]:
optimizer = torch.optim.SGD(net.parameters(), lr=0.1)
# class torch.optim.SGD(params, lr=, momentum=0, dampening=0, weight_decay=0, nesterov=False)

## 3.7.5 训练模型

In [10]:
num_epochs = 5
d2l.train_ch3(net, train_iter, test_iter, loss, num_epochs, batch_size, None, None, optimizer)

epoch 1, loss 0.0031, train acc 0.748, test acc 0.772
epoch 2, loss 0.0022, train acc 0.813, test acc 0.813
epoch 3, loss 0.0021, train acc 0.826, test acc 0.817
epoch 4, loss 0.0020, train acc 0.833, test acc 0.823
epoch 5, loss 0.0019, train acc 0.837, test acc 0.818
